In [1]:
from collections import defaultdict
import requests
import json
import time


base_url = 'https://api.github.com/search/issues?q=-label:invalid+type:pr+is:public+author:zhangkaihua88&per_page=1000'


def get_json(url):
    response = requests.get(url)
    data = json.loads(response.text)
    return data


raw_data = get_json(base_url)
raw_data = [item for item in raw_data['items'] if item['state'] == 'closed']
# print(data)

In [2]:
data = defaultdict(dict)

In [3]:
for item_data in raw_data:
    item_repo = item_data['repository_url'].split('/')[-1]
    item_name = item_data['repository_url'].split('/')[-2]

    item_repo_url = item_data['repository_url'].replace('api.', '').replace('repos/', '')
    item_name_url = item_data['repository_url'].replace('api.', '').replace('repos/', '').replace(f'/{item_repo}', '')

    item_data_appended = get_json(item_data['pull_request']['url'])
    if item_data_appended['merged']:
        add_base = data[f'{item_name}/{item_repo}'].get('additions', 0)
        del_base = data[f'{item_name}/{item_repo}'].get('deletions', 0)
        count_base = data[f'{item_name}/{item_repo}'].get('count', 0)
        data[f'{item_name}/{item_repo}'] = {
            'additions':item_data_appended['additions']+add_base, 
            'deletions':item_data_appended['deletions']+del_base,
            'count':count_base+1,
        }


In [6]:
keys = sorted(data, key=lambda x: data[x]['additions']+data[x]['deletions'], reverse=True)

rougier/scientific-visualization-book 12 2
dptech-corp/Uni-Mol 2 1
freeCodeCamp/how-to-contribute-to-open-source 2 1


In [25]:
def gen_item(data, key):
    strings = f"[{key}](https://github.com/{key}) PR " + str(data['count']) + " times with ${\color{green}{-" + str(data['deletions']) + "}} \ {\color{red}{+" + str(data['additions']) + "}}$\n"
    return strings

strings = '<img src="./images/git-merge.svg" width="60px" style="max-width: 100%;">\n'
for item in keys:
    strings += gen_item(data[item], item)

In [26]:
print(strings)

[rougier/scientific-visualization-book](https://github.com/rougier/scientific-visualization-book) PR 2 times with ${\color{green}{-6}} \ {\color{red}{+6}}$
[dptech-corp/Uni-Mol](https://github.com/dptech-corp/Uni-Mol) PR 1 times with ${\color{green}{-1}} \ {\color{red}{+1}}$
[freeCodeCamp/how-to-contribute-to-open-source](https://github.com/freeCodeCamp/how-to-contribute-to-open-source) PR 1 times with ${\color{green}{-1}} \ {\color{red}{+1}}$

